In [7]:
import pdb
import math
import random
import operator
import collections
import pandas as pd
import time


import mmh3

from IPython.display import display

# 0. Data
 - Scenario 2: [Botnet-43](https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-43/)
 - Infected host: 147.32.84.165 

In [8]:
def getData(filename):    
    cols = ['Date','Duration','Protocol','Source','Destination','Label']
    list = []
    with open(filename) as file:  
        for count, line in enumerate(file):
            if count!=0:
                data = line.split("\t")
                list.append([data[0],data[1],data[2],data[3].split(':')[0],data[5].split(':')[0],data[11].split('\n')[0]])
    
    dataset = pd.DataFrame(list, columns = cols)
    return dataset

if __name__ == "__main__":
    filename    = 'data/capture20110811.pcap.netflow.labeled'
    df_scene02  = getData(filename)
    df_scene02  = df_scene02[df_scene02['Source'] == '147.32.84.165']
    print (' - Total Rows : ', len(df_scene02))
    display(df_scene02.head(n=10))

 - Total Rows :  45261


,Date,Duration,Protocol,Source,Destination,Label
540283,2011-08-11 10:27:20.087,0.000,UDP,147.32.84.165,147.32.80.9,1
541362,2011-08-11 10:27:22.334,0.000,UDP,147.32.84.165,147.32.80.9,1
541377,2011-08-11 10:27:22.355,0.045,TCP,147.32.84.165,74.125.232.198,Botnet
702906,2011-08-11 10:32:25.092,0.000,UDP,147.32.84.165,147.32.80.9,1
703012,2011-08-11 10:32:25.294,0.126,TCP,147.32.84.165,195.113.232.98,Botnet
756366,2011-08-11 10:34:05.813,0.000,UDP,147.32.84.165,147.32.80.9,1
756532,2011-08-11 10:34:06.163,3.307,TCP,147.32.84.165,60.190.222.139,Botnet
761981,2011-08-11 10:34:15.379,0.000,TCP,147.32.84.165,60.190.222.139,Botnet
768197,2011-08-11 10:34:27.397,0.000,TCP,147.32.84.165,60.190.222.139,Botnet
784740,2011-08-11 10:34:57.700,3.244,TCP,147.32.84.165,60.190.222.139,Botnet


In [9]:
df_scene02.groupby(['Source', 'Label']).aggregate({'Protocol':'count'}).sort_values('Protocol', ascending=False)

Protocol
Source        Label            
147.32.84.165 Botnet      31222
              1           13096
              Botnet        943

# 1. Task
 - To-Build
     - Build code for computing a COUNT-MIN sketch
     - play with different heights and widths
 - To-Analyze
     - Compare it to the RESERVOIR sampling strategy(space- efficient/accurate, run-time analysis)
     - Use the theory to explain any differences you observe

# 2.1 Grouth Truth Data
 - Top 10 frequently connected IPs

In [10]:
def getTopIPs(df, k=10, verbose=0):
    df_globaltop10       = df.groupby('Destination')['Destination'].agg(['count']).nlargest(k, 'count')
    ip_globaltop10       = df_globaltop10.index.tolist()
    ip_globaltop10_count = df_globaltop10['count'].tolist()

    if verbose:
        for i,ip in enumerate(ip_globaltop10):
            count = ip_globaltop10_count[i]
            perc  = round(100.0* count / len(df),2) 
            print (' - IP : ', ip, ' || Netflows : ', count, ' || Perc : ', perc)
    
    return ip_globaltop10, ip_globaltop10_count

if __name__ == "__main__":
    ip_globaltopK, ip_globaltopK_count = getTopIPs(df_scene02, k=10, verbose=1)

 - IP :  193.23.181.44  || Netflows :  6167  || Perc :  13.63
 - IP :  174.128.246.102  || Netflows :  3427  || Perc :  7.57
 - IP :  174.37.196.55  || Netflows :  3357  || Perc :  7.42
 - IP :  67.19.72.206  || Netflows :  3135  || Perc :  6.93
 - IP :  72.20.15.61  || Netflows :  2966  || Perc :  6.55
 - IP :  173.236.31.226  || Netflows :  1708  || Perc :  3.77
 - IP :  184.154.89.154  || Netflows :  1678  || Perc :  3.71
 - IP :  46.4.36.120  || Netflows :  1627  || Perc :  3.59
 - IP :  147.32.80.9  || Netflows :  787  || Perc :  1.74
 - IP :  217.163.21.37  || Netflows :  701  || Perc :  1.55


# 2.2 [Count-Min Sketching](https://crahen.github.io/algorithm/stream/count-min-sketch-point-query.html)

In [13]:
class CMSketch:
    
    # Set size, width and depth parameters and sets array values as 0
    def __init__(self, w, d):
        self.w            = w
        self.count_hash   = d
        self.sketch_size  = w*d
        self.sketch_array = [[0] * w for i in range(d)]
    
    # To add IP passed as parameter to the sketch array
    def insert(self, string):
        for i in range(self.count_hash):
            result = mmh3.hash(string, i) % self.w
            self.sketch_array[i][result] += 1
    
    # To carry out a point query
    def point_query(self, string):
        min = 9999999
        for i in range(self.count_hash):
            result = mmh3.hash(string, i) % self.w
            if self.sketch_array[i][result] < min:
                min = self.sketch_array[i][result]
        return min        

# add product of w and d to show complexity.
def CM_Apply(df, infected_ip):
    
    IPs_globaltopK, IPs_globaltopK_count = getTopIPs(df, k=10, verbose=0)
    IPs = df['Destination'].tolist() 
    
    best_accuracy = 0
    best_w = 9999
    best_d = 9999
    best_complexity = 9999999
    
    e = 2.71828183 
    res = []
    for eps in [0.0001, 0.001, 0.005, 0.01, 0.1]:
        for delta in [0.0001, 0.001, 0.005, 0.01, 0.1, 0.2, 0.3]:
            start = time.time()
            #Determine width w and number of hash functions d
            w = round(e/eps)
            d = math.log(1/delta)
        
            #Create sketch w, d as dimensions
            CMSketchObj = CMSketch(int(w), int(d))
        
            # Add each IP to sketch
            for ip in IPs:
                CMSketchObj.insert(ip)
            
            #Finding frequency and storing it    
            cm = {}
            for ip in IPs:
                cm[ip] = CMSketchObj.point_query(ip)
            end = time.time()
            print('Time: ',end-start)
            # Sorting to find top 10 most frequent
            cm_sorted = sorted(cm.items(), key = operator.itemgetter(1), reverse = True)
            
            # Find difference between ground truth and oount-min sketch, tells you how accurately the countmin
            # sketch gets the top 10 IPs for different w,d.
            count = 0
            for i in range(10):
                if cm_sorted[i][0] == IPs_globaltopK[i]:
                    count += 1
                    
            accuracy   = round(count/10.0,3)
            complexity = round(w * d)
            print('For w      = ', w, 'and d = ',d)
            print('Complexity = ', w * d)
            print('Accuracy   = ', accuracy)
            print('Epsilon    = ', eps)
            print('Delta      = ', delta)
            # print('Difference = ', diff)
            
            if accuracy >= best_accuracy:
                best_accuracy = accuracy
                if complexity < best_complexity:
                    best_complexity = complexity
                    
                    print (' ---------------------------- ')
                    for i in range(10):
                        print ('%.12s || %.12s' % (cm_sorted[i][0], IPs_globaltopK[i]))
                    print (' ---------------------------- ')
            
            print('________________________________________')
            res.append([eps, delta, complexity, accuracy])
    
    return res
            
if __name__ == "__main__":
    res = CM_Apply(df_scene02, '147.32.84.165')
    df = pd.DataFrame(res, columns=['ε', 'δ', 'Complexity', 'Accuracy'])
    display(df)
    
    """
    For w      =  2718 and d =  5.298317366548036
    Complexity =  14401
    Accuracy   =  0.9
    Epsilon    =  0.001
    Delta      =  0.005
    """

Time:  0.35009193420410156
For w      =  27183 and d =  9.210340371976184
Complexity =  250364.6823314286
Accuracy   =  1.0
Epsilon    =  0.0001
Delta      =  0.0001
 ---------------------------- 
193.23.181.4 || 193.23.181.4
174.128.246. || 174.128.246.
174.37.196.5 || 174.37.196.5
67.19.72.206 || 67.19.72.206
72.20.15.61 || 72.20.15.61
173.236.31.2 || 173.236.31.2
184.154.89.1 || 184.154.89.1
46.4.36.120 || 46.4.36.120
147.32.80.9 || 147.32.80.9
217.163.21.3 || 217.163.21.3
 ---------------------------- 
________________________________________
Time:  0.22635626792907715
For w      =  27183 and d =  6.907755278982137
Complexity =  187773.5117485714
Accuracy   =  1.0
Epsilon    =  0.0001
Delta      =  0.001
 ---------------------------- 
193.23.181.4 || 193.23.181.4
174.128.246. || 174.128.246.
174.37.196.5 || 174.37.196.5
67.19.72.206 || 67.19.72.206
72.20.15.61 || 72.20.15.61
173.236.31.2 || 173.236.31.2
184.154.89.1 || 184.154.89.1
46.4.36.120 || 46.4.36.120
147.32.80.9 || 147.32.8

Time:  0.10870838165283203
For w      =  544 and d =  2.302585092994046
Complexity =  1252.6062905887609
Accuracy   =  0.8
Epsilon    =  0.005
Delta      =  0.1
________________________________________
Time:  0.07280111312866211
For w      =  544 and d =  1.6094379124341003
Complexity =  875.5342243641505
Accuracy   =  0.1
Epsilon    =  0.005
Delta      =  0.2
________________________________________
Time:  0.07280778884887695
For w      =  544 and d =  1.2039728043259361
Complexity =  654.9612055533092
Accuracy   =  0.1
Epsilon    =  0.005
Delta      =  0.3
________________________________________
Time:  0.35201406478881836
For w      =  272 and d =  9.210340371976184
Complexity =  2505.2125811775218
Accuracy   =  1.0
Epsilon    =  0.01
Delta      =  0.0001
________________________________________
Time:  0.2304232120513916
For w      =  272 and d =  6.907755278982137
Complexity =  1878.9094358831412
Accuracy   =  1.0
Epsilon    =  0.01
Delta      =  0.001
 ----------------------------

,ε,δ,Complexity,Accuracy
0,0.0001,0.0001,250365,1.0
1,0.0001,0.0010,187774,1.0
2,0.0001,0.0050,144024,1.0
3,0.0001,0.0100,125182,1.0
4,0.0001,0.1000,62591,1.0
5,0.0001,0.2000,43749,1.0
6,0.0001,0.3000,32728,1.0
7,0.0010,0.0001,25034,1.0
8,0.0010,0.0010,18775,1.0
9,0.0010,0.0050,14401,1.0


In [6]:
#Space used: O((1 / ε) * ln(1 / δ))
#Time Needed = O(ln(1 / δ))

# Cannot have delta greater than 0.3 for epsilon values as listed. Depth cannot be lowered for 
# fixed epsilon values. Width has to be increased appropriately for reduction in depth.
